In [1]:
import cv2
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from pyzbar.pyzbar import decode
from pdf2image import convert_from_path
from io import BytesIO
from PIL import Image
from bs4 import BeautifulSoup
from selenium import webdriver
from pdf2image import convert_from_path
from IPython.display import display, Image
import pyautogui
import time

In [3]:
# os.system('explorer')
# os.startfile("D:\PREVALENTWARE\RAPPI\lectura-csf-mexico\CFS ADBID LATAM MX SA DE CV.pdf")
# time.sleep(5)
# screenshot = pyautogui.screenshot()
# screenshot.save(r'D:\PREVALENTWARE\RAPPI\lectura-csf-mexico\filename.png')

Obtengo los archivos PDF para convertir a Imagen

In [2]:
directory = os.getcwd()

print(directory)
files_pdf = []
with os.scandir(path=directory) as itr:
    for file in itr:
        if file.name.split('.')[-1]=="pdf":
            files_pdf.append(file.name)
    print(files_pdf)        

d:\PREVALENTWARE\RAPPI\lectura-csf-mexico
['CSF MARIA DEL ROCIO.pdf', 'CSF SERVICIOS JURIDICOS Y ASESORIA POLITICA INTEGRAL.pdf', 'CSF VECH SOLUCIONES EN TRANSPORTE.pdf']


In [3]:
os.system('explorer')
for file in files_pdf:
    file_name_png = file.split('.')[0] + ".png"
    print(file_name_png)

    path = r"D:\PREVALENTWARE\RAPPI\lectura-csf-mexico\%s" % file
    os.startfile("D:\\PREVALENTWARE\\RAPPI\\lectura-csf-mexico\\%s" % file)
    time.sleep(3)
    screenshot = pyautogui.screenshot()
    screenshot.save("D:\\PREVALENTWARE\\RAPPI\\lectura-csf-mexico\\%s" % file_name_png)

   
    # # except OSError:
    # #     print(OSError)
    # # else:
    # #     print("Se creo el screen")
# os.system("taskkill /im chrome.exe /f")

CSF MARIA DEL ROCIO.png
CSF SERVICIOS JURIDICOS Y ASESORIA POLITICA INTEGRAL.png
CSF VECH SOLUCIONES EN TRANSPORTE.png


Obtengo los archivos imagenes

In [4]:
files_png = []
with os.scandir(path=directory) as itr:
    for file in itr:
        if file.name.split('.')[-1]=="png":
            files_png.append(file.name)
    print(files_png)   

['CSF MARIA DEL ROCIO.png', 'CSF SERVICIOS JURIDICOS Y ASESORIA POLITICA INTEGRAL.png', 'CSF VECH SOLUCIONES EN TRANSPORTE.png']


Obtener las URL de cada archivo png

In [5]:
urls = []
for file in files_png:
    img = cv2.imread("D:\\PREVALENTWARE\\RAPPI\\lectura-csf-mexico\\%s" % file)
    for code in decode(img):
        if(code.type == 'QRCODE'):
            urls.append(code.data.decode('utf-8'))
print(urls)

['https://siat.sat.gob.mx/app/qr/faces/pages/mobile/validadorqr.jsf?D1=10&D2=1&D3=14061187505_BACR6608146T0']


Ingreso a cada URL para capturar la data

In [6]:
list_heads = []
list_rows = []
for url in urls:
    time.sleep(1)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_window_size(1200, 1000)
    driver.get(url)
    time.sleep(1)

    table1 = driver.find_element(
                By.XPATH,'//*[@id="ubicacionForm:j_idt10:0:j_idt11:j_idt14_data"]')

    table1 = table1.text.split('\n')
    for row in table1:
        row1 = row.split(':')[0]
        row2 = row.split(':')[1]
        list_heads.append(row1)
        list_rows.append(row2)

    table2 = driver.find_element(
                    By.XPATH,'//*[@id="ubicacionForm:j_idt10:1:j_idt11:j_idt14_data"]')

    table2 = table2.text.split('\n')
    for row in table2:
        row1 = row.split(':')[0]
        row2 = row.split(':')[1]
        list_heads.append(row1)
        list_rows.append(row2)

    table3 = driver.find_element(
                    By.XPATH,'//*[@id="ubicacionForm:j_idt10:2:j_idt11:j_idt14_data"]')

    table3 = table3.text.split('\n')
    for row in table3:
        row1 = row.split(':')[0]
        row2 = row.split(':')[1]
        list_heads.append(row1)
        list_rows.append(row2)

print(list_heads)
print(list_rows)




['CURP', 'Nombre', 'Apellido Paterno', 'Apellido Materno', 'Fecha Nacimiento', 'Fecha de Inicio de operaciones', 'Situación del contribuyente', 'Fecha del último cambio de situación', 'Entidad Federativa', 'Municipio o delegación', 'Colonia', 'Tipo de vialidad', 'Nombre de la vialidad', 'Número exterior', 'Número interior', 'CP', 'Correo electrónico', 'AL', 'Régimen', 'Fecha de alta']
[' BACR660814MDFRSC05', ' MARIA DEL ROCIO', ' BARRIOS', ' CASTILLO', ' 14-08-1966', ' 19-07-2006', ' ACTIVO', ' 19-07-2006', ' MEXICO', ' NAUCALPAN DE JUAREZ', ' JARDINES DE SATELITE', ' CALLE', ' VIA LACTEA', ' 275', ' MZ. 10 VIV. A', ' 53129', ' jhopo_3@yahoo.com.mx', ' MEXICO 2', ' Régimen de las Personas Físicas con Actividades Empresariales y Profesionales', ' 19-07-2006']


Data Frame

In [7]:
df = pd.DataFrame({'Informacion':list_heads, 'datos':list_rows})
df.to_excel('file.xlsx', index = False, header = False, encoding ='utf-8')

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
